In [ ]:
from google.colab import drive

MOUNTPOINT = '/content/gdrive'
drive.mount(MOUNTPOINT)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import glob
import sklearn.model_selection

input_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC2018_Task1-2_Training_Data/ISIC2018_Task1-2_Training_Input_x2/*.jpg'
output_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC2018_Task1-2_Training_Data/ISIC2018_Task1_Training_GroundTruth_x2/*.png'

input = sorted(glob.glob(input_dataset))
output = sorted(glob.glob(output_dataset))

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(input, output, test_size=0.1) #potentially should lower test size
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=0.111) 

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))

train_ds = train_ds.shuffle(len(x_train))
test_ds = test_ds.shuffle(len(x_test))
val_ds = val_ds.shuffle(len(x_val))


def process(input_path, output_path):
  img = tf.image.decode_jpeg(tf.io.read_file(input_path), channels=3)
  img = tf.image.resize(img, [256,256])
  img = tf.cast(img, tf.float32) / 255 #normalize
  img = tf.reshape(img, (-1, 256, 256, 3)) # resize

  mask = tf.image.decode_png(tf.io.read_file(output_path), channels=1)
  mask = tf.image.resize(mask, [256,256])
  mask = tf.math.round(tf.cast(mask, tf.float32) / 255)
  mask = tf.reshape(mask, (-1, 256, 256, 1))

  return img, mask

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)
val_ds = val_ds.map(process)

#y_test is empty for some reason

In [ ]:
# If you are using Google Colab -> copy paste contents of model.py into here OR upload model.py into Google Drive
# If you are using local PC -> import model 

In [ ]:
height = 256
width = 256
channels = 3

def dice_coefficient(y_pred, y_test):
    flat_pred = tf.keras.backend.flatten(y_pred)
    flat_test = tf.keras.backend.flatten(y_test)
    return 2 * tf.keras.backend.sum(flat_pred * flat_test) / (tf.keras.backend.sum(flat_pred) + tf.keras.backend.sum(flat_test)) #2 * intersection / union

def dice_coefficient_loss(y_pred, y_test):
    return 1 - dice_coefficient(y_pred, y_test)

unet = model(height, width, channels)
unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 5*(10**-4)), loss=['binary_crossentropy'], metrics=["accuracy", dice_coefficient])
unet.summary()

history =  unet.fit(train_ds, validation_data=val_ds, batch_size=256, epochs=20)


In [ ]:
#calc dice coefficient between predict_test and test_y

total = 0

for img, mask in test_ds:
  pred = unet.predict(img)
  total += dice_coefficient(pred, mask)

dice_coefficient = total / len(predict_test)

# Dice coefficient: 
tf.print(dice_coefficient, summarize=-1)


In [ ]:
# plot loss function and accuracy